In [1]:
import config
API_KEY = config.API_KEY
API_SECRET = config.API_SECRET

https://stackoverflow.com/questions/40023931/how-to-get-real-estate-data-with-idealista-api

In [2]:
import pandas as pd
import json
import urllib
import requests as rq
import base64

In [3]:
def get_oauth_token(): 
    url = "https://api.idealista.com/oauth/token"    
    apikey= API_KEY #sent by idealist
    secret= API_SECRET #sent by idealista
    apikey_secret = apikey + ':' + secret
    auth = str(base64.b64encode(bytes(apikey_secret, 'utf-8')))[2:][:-1]
    headers = {'Authorization' : 'Basic ' + auth,'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}
    params = urllib.parse.urlencode({'grant_type':'client_credentials'}) #,'scope':'read'
    content = rq.post(url,headers = headers, params=params)
    bearer_token = json.loads(content.text)['access_token']
    return bearer_token

In [4]:
def search_api(token, URL):  
    headers = {'Content-Type': 'Content-Type: multipart/form-data;', 'Authorization' : 'Bearer ' + token}
    content = rq.post(url, headers = headers)
    result = json.loads(content.text)
    return result

In [5]:
# values: es, it, pt
country = 'es' 
# search language for summary. values: es, it, pt, en, ca
locale = 'es' 
language = 'es' 
# items per page: 50 as maximum allowed
max_items = '50'
# values: sale, rent
operation = 'sale'
# values: homes, offices, premises, garages, bedrooms
property_type = 'homes'
# allowed values by property type:
# garages: distance, price, street, photos, publicationDate, modificationDate (rent only), weigh, pricedown
# premises: distance, price, street, photos, publicationDate, modificationDate, size, floor, ratioeurm2 (rent only), weigh, pricedown
# offices: distance, price, street, photos, publicationDate, modificationDate, size, floor, ratioeurm2, weigh, pricedown
# homes: distance, price, street, photos, publicationDate, modificationDate, size, floor, rooms, ratioeurm2 (sólo alquiler), weigh, pricedown
# rooms: distance, price, street, photos, publicationDate, modificationDate, floor
order = 'priceDown' 
# geographic coordinates (WGS84) (latitude,longitude)
center = '43.3773,-8.39852' 
# distance to center, in metres (ratio)
distance = '3000'
# values: asc, desc
sort = 'desc'

In [6]:
url = ('https://api.idealista.com/3.5/'+country+'/search?operation='+operation+#"&locale="+locale+
           '&maxItems='+max_items+
           '&order='+order+
           '&center='+center+
           '&distance='+distance+
           '&propertyType='+property_type+
           '&sort='+sort+ 
           '&numPage=%s'+
           '&language='+language)

In [7]:
url

'https://api.idealista.com/3.5/es/search?operation=sale&maxItems=50&order=priceDown&center=43.3773,-8.39852&distance=3000&propertyType=homes&sort=desc&numPage=%s&language=es'

In [8]:
result = search_api(get_oauth_token(), url)

In [9]:
result

{'elementList': [{'propertyCode': '36194957',
   'thumbnail': 'https://img3.idealista.com/blur/WEB_LISTING/0/id.pro.es.image.master/f8/3a/9c/1035870049.jpg',
   'numPhotos': 34,
   'floor': '3',
   'price': 600000.0,
   'propertyType': 'flat',
   'operation': 'sale',
   'size': 180.0,
   'exterior': True,
   'rooms': 5,
   'bathrooms': 3,
   'address': 'barrio Ensanche',
   'province': 'A Coruña',
   'municipality': 'A Coruña',
   'district': 'Ensanche - Juan Florez',
   'country': 'es',
   'neighborhood': 'Ensanche',
   'latitude': 43.3642003,
   'longitude': -8.4047906,
   'showAddress': False,
   'url': 'https://www.idealista.com/inmueble/36194957/',
   'distance': '1542',
   'description': 'Magnífico piso en el centro de La Coruña en un edificio de 1960. A la fachada tiene tres estancias, salón con un balcón, comedor y dormitorio principal. Al patio de luces da la cocina con office, zona de lavandería y cuarto de plancha, un cuarto de baño completo y tres dormitorios y al patio de 

In [10]:
with open('idealista_search.txt', 'w') as convert_file:
     convert_file.write(json.dumps(result))

In [11]:
type(result)

dict